# Create monthly CESM atmospheric dust and black carbon deposition files

Interpolated to ANHA12 model grid.

NCAR CESM output:
Community Earth System Model (Community Atmosphere Model - CAM https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2013MS000279) output: https://www.earthsystemgrid.org/; CESM2 Large Ensemble Atmosphere Post Processed Data, Monthly Averages.

Specific run output: https://www.earthsystemgrid.org/dataset/ucar.cgd.cesm2le.output.html

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import sys
sys.path.append('../paper-materials/')
import mapping_functions as mf
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Parameters

In [2]:
data_folder = '/data/brogalla/NCAR-CESM2/'

#### Load files

In [3]:
# ANHA12 grid mesh:
mesh        = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask       = mesh['tmask'].values[0,:,:,:]
ANHA12_lons = mesh['nav_lon']
ANHA12_lats = mesh['nav_lat']
Z_masked    = np.ma.masked_where((tmask > 0.1), tmask) 

In [ ]:
# ---- Load data: ------------
# From CESM2 experiment linked at top of this notebook.
# Black carbon:
BC0 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_BLACK_CARBON_FLUX_CPL.199001-199912.nc')
BC1 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_BLACK_CARBON_FLUX_CPL.200001-200912.nc')
BC2 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_BLACK_CARBON_FLUX_CPL.201001-201412.nc')
BC3 = xr.open_dataset(f'{data_folder}b.e21.BSSP370cmip6.f09_g17.LE2-1001.001.pop.h.ATM_BLACK_CARBON_FLUX_CPL.201501-202412.nc')
ATM_BC0  = BC0['ATM_BLACK_CARBON_FLUX_CPL'].values
ATM_BC1  = BC1['ATM_BLACK_CARBON_FLUX_CPL'].values # time, lat, lon; g/cm2/s
ATM_BC2  = BC2['ATM_BLACK_CARBON_FLUX_CPL'].values
ATM_BC3  = BC3['ATM_BLACK_CARBON_FLUX_CPL'].values
BC0_time = BC0.indexes['time'].to_datetimeindex() 
BC1_time = BC1.indexes['time'].to_datetimeindex() 
BC2_time = BC2.indexes['time'].to_datetimeindex() 
BC3_time = BC3.indexes['time'].to_datetimeindex() 
# combine:
ATM_BC   = np.vstack([ATM_BC1, ATM_BC2, ATM_BC3])
BC_time  = np.hstack([BC1_time, BC2_time, BC3_time])
BC_date  = np.array([pd.Timestamp(time).to_pydatetime() for time in BC_time])
  
# Fine dust:
FD1 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_FINE_DUST_FLUX_CPL.200001-200912.nc')
FD2 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_FINE_DUST_FLUX_CPL.201001-201412.nc')
FD3 = xr.open_dataset(f'{data_folder}b.e21.BSSP370cmip6.f09_g17.LE2-1001.001.pop.h.ATM_FINE_DUST_FLUX_CPL.201501-202412.nc')
ATM_FD1  = FD1['ATM_FINE_DUST_FLUX_CPL'].values # time, lat, lon; g/cm2/s
ATM_FD2  = FD2['ATM_FINE_DUST_FLUX_CPL'].values
ATM_FD3  = FD3['ATM_FINE_DUST_FLUX_CPL'].values
FD1_time = FD1.indexes['time'].to_datetimeindex() 
FD2_time = FD2.indexes['time'].to_datetimeindex() 
FD3_time = FD3.indexes['time'].to_datetimeindex()
# combine:
ATM_FD   = np.vstack([ATM_FD1, ATM_FD2, ATM_FD3])
FD_time  = np.hstack([FD1_time, FD2_time, FD3_time])
FD_date  = np.array([pd.Timestamp(time).to_pydatetime() for time in FD_time])
    
# Coarse dust: 
CD1 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_COARSE_DUST_FLUX_CPL.200001-200912.nc')
CD2 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.ATM_COARSE_DUST_FLUX_CPL.201001-201412.nc')
CD3 = xr.open_dataset(f'{data_folder}b.e21.BSSP370cmip6.f09_g17.LE2-1001.001.pop.h.ATM_COARSE_DUST_FLUX_CPL.201501-202412.nc')
ATM_CD1  = CD1['ATM_COARSE_DUST_FLUX_CPL'].values # time, lat, lon; g/cm2/s
ATM_CD2  = CD2['ATM_COARSE_DUST_FLUX_CPL'].values
ATM_CD3  = CD3['ATM_COARSE_DUST_FLUX_CPL'].values
CD1_time = CD1.indexes['time'].to_datetimeindex() 
CD2_time = CD2.indexes['time'].to_datetimeindex() 
CD3_time = CD3.indexes['time'].to_datetimeindex()
# combine:
ATM_CD   = np.vstack([ATM_CD1, ATM_CD2, ATM_CD3])
CD_time  = np.hstack([CD1_time, CD2_time, CD3_time])
CD_date  = np.array([pd.Timestamp(time).to_pydatetime() for time in CD_time])
    
# Latitudes and longitudes of T grid points
CESM2_lons = BC1['TLONG'].values
CESM2_lons[CESM2_lons >=180] = -360 + CESM2_lons[CESM2_lons >=180] # degrees east
CESM2_lats = BC1['TLAT'].values

In [5]:
# Function to save CESM2 monthly output fields that are read in by NEMO
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # convert units from g/cm2/s to kg/m2/s: (1 kg/ 1000 g) * (10,000 cm2 / m2)
    field1_convert = field1*10
    field2_convert = field2*10
    field3_convert = field3*10
    
    # variables
    fine_dust             = ncd.createVariable('fdust', 'float64', ('y','x'))
    fine_dust.units       = 'kg/m2/s'
    fine_dust.long_name   = 'Fine dust deposition flux'  
    fine_dust.coordinates = 'nav_lon nav_lat'
    fine_dust[:]          = field1_convert
    
    coarse_dust             = ncd.createVariable('cdust', 'float64', ('y','x'))
    coarse_dust.units       = 'kg/m2/s'
    coarse_dust.long_name   = 'Coarse dust deposition flux'  
    coarse_dust.coordinates = 'nav_lon nav_lat'
    coarse_dust[:]          = field2_convert
    
    black_c             = ncd.createVariable('bc', 'float64', ('y','x'))
    black_c.units       = 'kg/m2/s'
    black_c.long_name   = 'Black carbon deposition flux'  
    black_c.coordinates = 'nav_lon nav_lat'
    black_c[:]          = field3_convert
    
    print('saved ', filename)

    ncd.close()
    return

In [6]:
# Function identifies for a given file year what the indices are for those dates within the CESM2 files
def find_dates(file_year):
    
    indices = np.array([])
    for d,date in enumerate(FD_date):
        if date.year==file_year:
            indices = np.append(indices, d)
            
    start_index = int(np.amin(indices))
    end_index   = int(np.amax(indices))
    
    print('start index: ', start_index)
    print('end index: ', end_index)
    
    return start_index, end_index

In [7]:
# Main function that identifies the dates for the particular file year, interpolates the file to the ANHA12 grid, and saves the file
def CESM_to_ANHA12(file_year, savefiles=False):
    
    si, ei = find_dates(file_year)
    
    # Select dates
    fine_dust    = ATM_FD[si:ei+1,:,:]
    coarse_dust  = ATM_CD[si:ei+1,:,:]
    black_carbon = ATM_BC[si:ei+1,:,:]
    
    print('Check that output is of the correct dimensions: ', fine_dust.shape, coarse_dust.shape, black_carbon.shape)
    
    interp_fine_dust    = np.empty((12, 2400, 1632))
    interp_coarse_dust  = np.empty((12, 2400, 1632))
    interp_black_carbon = np.empty((12, 2400, 1632))

    # loop over the months:
    for i in range(0,12):
        # Mask NaN values and remove
        fdm = np.ma.masked_where(np.isnan(fine_dust[i,:,:]), fine_dust[i,:,:])
        cdm = np.ma.masked_where(np.isnan(coarse_dust[i,:,:]), coarse_dust[i,:,:])
        bcm = np.ma.masked_where(np.isnan(black_carbon[i,:,:]), black_carbon[i,:,:])
        
        # Interpolate
        interp_fine_dust[i,:,:]    = mf.interp_np(CESM2_lons[~fdm.mask], CESM2_lats[~fdm.mask], \
                                                  fdm[~fdm.mask], ANHA12_lons, ANHA12_lats)
        interp_coarse_dust[i,:,:]  = mf.interp_np(CESM2_lons[~cdm.mask], CESM2_lats[~cdm.mask], \
                                                  cdm[~cdm.mask], ANHA12_lons, ANHA12_lats)
        interp_black_carbon[i,:,:] = mf.interp_np(CESM2_lons[~bcm.mask], CESM2_lats[~bcm.mask], \
                                                  bcm[~bcm.mask], ANHA12_lons, ANHA12_lats)
        
    if savefiles:
        location = '/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/atmospheric/'
        
        for ind in range(1,13):
            save_file(f'{location}new-atm_flux_y{file_year}m{ind:02}.nc',interp_fine_dust[ind-1,:,:],\
                      interp_coarse_dust[ind-1,:,:], interp_black_carbon[ind-1,:,:])
    
    return interp_fine_dust, interp_coarse_dust, interp_black_carbon

#### Interpolate to ANHA12 grid:

In [11]:
ATM_FDm = np.ma.masked_where(np.isnan(ATM_FD), ATM_FD)
ATM_CDm = np.ma.masked_where(np.isnan(ATM_CD), ATM_CD)
ATM_BCm = np.ma.masked_where(np.isnan(ATM_BC), ATM_BC)

In [ ]:
for year in np.arange(2002,2021,1):
    interp_fine_dust, interp_coarse_dust, interp_black_carbon = CESM_to_ANHA12(year, savefiles=True)